In [1]:
import cv2
from PIL import Image
import numpy as np
from datasets import load_dataset

train = load_dataset("JotDe/birds", split="train")
test = load_dataset("JotDe/birds", split="test")


c:\Users\emerc\anaconda3\envs\video\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (C:/Users/emerc/.cache/huggingface/datasets/JotDe___parquet/JotDe--birds-c415c99c78c5e8f0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/emerc/.cache/huggingface/datasets/JotDe___parquet/JotDe--birds-c415c99c78c5e8f0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [2]:
xtrain = train['image'][0:88]
ytrain = train['label'][0:88]
xtest = test['image'][0:88]
ytest = test['label'][0:88]

list = [xtrain, xtest]

In [50]:
print(xtrain.head(5))

AttributeError: 'list' object has no attribute 'head'

In [17]:
import pandas as pd
new_x_train = pd.DataFrame()
new_x_test = pd.DataFrame()
for image in xtrain:
    numpy_image = np.array(image)
    # img1 = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2GRAY)
    orb = cv2.ORB_create()

    kp1 , des1 = orb.detectAndCompute(numpy_image, None)
    flattened = des1.flatten()
    flattened = flattened[0:4000]
    cmatrix = pd.DataFrame(flattened)
    cmatrix = cmatrix.transpose()    
    new_x_train = pd.concat([new_x_train, cmatrix],ignore_index=True)
    print(new_x_train.head(5))

for image in xtest:
    numpy_image = np.array(image)
    # img1 = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2GRAY)
    orb = cv2.ORB_create()

    kp1 , des1 = orb.detectAndCompute(numpy_image, None)
    flattened = des1.flatten()
    flattened = flattened[0:4000]
    cmatrix = pd.DataFrame(flattened)
    cmatrix = cmatrix.transpose()    
    new_x_test = pd.concat([new_x_test, cmatrix],ignore_index=True)
    print(new_x_test.head(5))



   0     1     2     3     4     5     6     7     8     9     ...  3990  \
0   250   122   105   125    60   217   209   182    51   236  ...   249   

   3991  3992  3993  3994  3995  3996  3997  3998  3999  
0     8   131   255   225    20   234   119    82    98  

[1 rows x 4000 columns]
   0     1     2     3     4     5     6     7     8     9     ...  3990  \
0   250   122   105   125    60   217   209   182    51   236  ...   249   
1   185   222   111   254    46   145   207   246   213    66  ...    82   

   3991  3992  3993  3994  3995  3996  3997  3998  3999  
0     8   131   255   225    20   234   119    82    98  
1   211   157   148    30   127   208    12    11    15  

[2 rows x 4000 columns]
   0     1     2     3     4     5     6     7     8     9     ...  3990  \
0   250   122   105   125    60   217   209   182    51   236  ...   249   
1   185   222   111   254    46   145   207   246   213    66  ...    82   
2   246   132   240   241   189    44   212   143 

In [18]:
print(new_x_train.shape)
new_y_train = pd.DataFrame(ytrain)
new_y_test = pd.DataFrame(ytest)
print(new_y_train.shape)
print(new_x_test.shape)

(88, 4000)
(88, 1)
(88, 4000)


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc, classification_report

#Instantiates the models
knc = KNeighborsClassifier(n_neighbors=3)
svc = SVC(probability=True)
nb = GaussianNB()
rfc = RandomForestClassifier()

### Fits the models ###
knc = knc.fit(new_x_train, new_y_train)
svc = svc.fit(new_x_train, new_y_train)
nb = nb.fit(new_x_train, new_y_train)
rfc = rfc.fit(new_x_train,new_y_train)

### Predicts test values ###
y_test_predict_knc = knc.predict(new_x_test)
y_test_predict_svc = svc.predict(new_x_test)
y_test_predict_nb = nb.predict(new_x_test)
y_test_predict_rfc = rfc.predict(new_x_test)

### Classification Reports
print(classification_report(new_y_test, y_test_predict_knc, zero_division = 0))
print(classification_report(new_y_test, y_test_predict_svc, zero_division = 0))
print(classification_report(new_y_test, y_test_predict_nb, zero_division = 0))
print(classification_report(new_y_test, y_test_predict_rfc, zero_division = 0))

### Predictions for testing values ###
y_test_predict_proba_knc = knc.predict_proba(new_x_test)
y_test_predict_proba_svc = svc.predict_proba(new_x_test)
y_test_predict_proba_nb = nb.predict_proba(new_x_test)
y_test_predict_proba_rfc = rfc.predict_proba(new_x_test)

c:\Users\emerc\anaconda3\envs\video\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\emerc\anaconda3\envs\video\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\emerc\anaconda3\envs\video\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\emerc\AppData\Local\Temp\ipykernel_20072\1283629100.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sh

              precision    recall  f1-score   support

           0       0.38      0.40      0.39        30
           1       0.19      0.13      0.16        30
           2       0.29      0.36      0.32        28

    accuracy                           0.30        88
   macro avg       0.28      0.30      0.29        88
weighted avg       0.28      0.30      0.29        88

              precision    recall  f1-score   support

           0       0.37      0.67      0.48        30
           1       0.35      0.40      0.38        30
           2       0.00      0.00      0.00        28

    accuracy                           0.36        88
   macro avg       0.24      0.36      0.28        88
weighted avg       0.25      0.36      0.29        88

              precision    recall  f1-score   support

           0       0.32      0.33      0.33        30
           1       0.24      0.20      0.22        30
           2       0.16      0.18      0.17        28

    accuracy        